In [7]:
import os
import json
import base64
from PIL import Image
from io import BytesIO

def convert_labelme_to_yolo(json_dir, output_dir, class_mapping):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(json_dir):
        if filename.endswith('.json'):
            json_path = os.path.join(json_dir, filename)
            with open(json_path, 'r') as f:
                data = json.load(f)

            image_data = data['imageData']
            image_height = data['imageHeight']
            image_width = data['imageWidth']
            
            shapes = data['shapes']
            yolo_labels = []

            for shape in shapes:
                label_name = shape['label']
                if label_name not in class_mapping:
                    continue

                label = class_mapping[label_name]
                
                if shape['shape_type'] == 'rectangle':
                    points = shape['points']
                    x_min, y_min = points[0]
                    x_max, y_max = points[1]
                    
                    # Convert to YOLO format
                    x_center = (x_min + x_max) / 2.0 / image_width
                    y_center = (y_min + y_max) / 2.0 / image_height
                    width = (x_max - x_min) / image_width
                    height = (y_max - y_min) / image_height

                    yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

                elif shape['shape_type'] == 'polygon':
                    points = shape['points']
                    x_min = min([point[0] for point in points])
                    x_max = max([point[0] for point in points])
                    y_min = min([point[1] for point in points])
                    y_max = max([point[1] for point in points])

                    # Convert to YOLO format
                    x_center = (x_min + x_max) / 2.0 / image_width
                    y_center = (y_min + y_max) / 2.0 / image_height
                    width = (x_max - x_min) / image_width
                    height = (y_max - y_min) / image_height

                    yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            # Write YOLO label file
            filename_stem = os.path.splitext(filename)[0]
            yolo_path = os.path.join(output_dir, filename_stem + '.txt')
            with open(yolo_path, 'w') as f:
                f.write('\n'.join(yolo_labels))

            # Decode base64 image data and save as image file
            image_data = base64.b64decode(image_data)
            image = Image.open(BytesIO(image_data))
            image_output_path = os.path.join(output_dir, filename_stem + '.jpg')
            image.save(image_output_path)

# Usage
data_rel_path = '../../data'.replace('/', os.path.sep)
data_abs_path = os.path.abspath(data_rel_path)
json_dir = os.path.join(data_abs_path, 'adaptive_thres_jsons')  # Replace with the path to your JSON files directory
output_dir = os.path.join(data_abs_path, 'dataset')    # Replace with the path to your output directory

# Define class mapping
class_mapping = {
    "arch": 0,
    "vomer": 1
}

convert_labelme_to_yolo(json_dir, output_dir, class_mapping)

In [8]:
# Validate the txt and jpg datasets have identical names. Later copy them into train and val folders

import os
import shutil
from sklearn.model_selection import train_test_split

def organize_dataset(dataset_dir, output_dir, split_ratio=0.2):
    # Check if output directories exist, if not, create them
    images_train_dir = os.path.join(output_dir, 'images/train')
    images_val_dir = os.path.join(output_dir, 'images/val')
    labels_train_dir = os.path.join(output_dir, 'labels/train')
    labels_val_dir = os.path.join(output_dir, 'labels/val')

    for dir_path in [images_train_dir, images_val_dir, labels_train_dir, labels_val_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    # Get lists of all jpg and txt files
    jpg_files = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg')]
    txt_files = [f for f in os.listdir(dataset_dir) if f.endswith('.txt')]

    # Check for 1-to-1 correspondence
    jpg_file_names = set(os.path.splitext(f)[0] for f in jpg_files)
    txt_file_names = set(os.path.splitext(f)[0] for f in txt_files)
    common_file_names = jpg_file_names.intersection(txt_file_names)

    if not (jpg_file_names == txt_file_names):
        print("Warning: Some jpg and txt files do not have corresponding pairs.")

    # Prepare paired file paths
    jpg_files = [os.path.join(dataset_dir, f"{name}.jpg") for name in common_file_names]
    txt_files = [os.path.join(dataset_dir, f"{name}.txt") for name in common_file_names]

    # Split data into training and validation sets
    jpg_train, jpg_val, txt_train, txt_val = train_test_split(
        jpg_files, txt_files, test_size=split_ratio, random_state=42
    )

    # Copy files to their respective directories
    def copy_files(files, output_dir):
        for file in files:
            shutil.copy(file, output_dir)

    copy_files(jpg_train, images_train_dir)
    copy_files(jpg_val, images_val_dir)
    copy_files(txt_train, labels_train_dir)
    copy_files(txt_val, labels_val_dir)

    print(f"Organized dataset into {images_train_dir}, {images_val_dir}, {labels_train_dir}, and {labels_val_dir}")

# Usage
data_rel_path = '../../data/dataset'.replace('/', os.path.sep)
data_abs_path = os.path.abspath(data_rel_path)
output_dir = os.path.join(data_abs_path, 'detect_arch')    # Replace with the path to your output directory

organize_dataset(data_abs_path, output_dir)

Organized dataset into d:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\images/train, d:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\images/val, d:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels/train, and d:\Code\CBCT_Reconstruction_ICP\data\dataset\detect_arch\labels/val


In [2]:
import os
print(os.environ['CONDA_PREFIX'])
try:
    import sklearn
    print("scikit-learn is installed.")
    print("scikit-learn version:", sklearn.__version__)
except ImportError:
    print("scikit-learn is not installed.")

C:\Users\DYPro\.conda\envs\mesh_seg_net
scikit-learn is installed.
scikit-learn version: 1.5.0


In [5]:
!conda install scikit-learn -y

Channels:
 - conda-forge
 - defaults
 - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
 - nvidia
 - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free
 - pytorch
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



In [1]:
import sys
print(sys.executable)

C:\Users\DYPro\.conda\envs\mesh_seg_net\python.exe


In [11]:
# Train Yolov10 model for the first time
import os
import torch
from ultralytics import YOLOv10

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Create a new YOLO model from scratch
model = YOLOv10('yolov10n.yaml')

# Ensure the model uses GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("using device: ", device)
model.to(device)

# Train the model using the 'arch_detection.yaml' dataset for 50 epochs
results = model.train(data='detech_arch.yaml', epochs=100, device=device)


using device:  cuda
New https://pypi.org/project/ultralytics/8.2.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov10n.yaml, data=arch_detection.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=cuda, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes

train: Scanning D:\Code\CBCT_Reconstruction_ICP\data\dataset\count_arch\labels\train... 124 images, 0 backgrounds, 0 corrupt: 100%|██████████| 124/124 [00:00<00:00, 1172.70it/s]

train: New cache created: D:\Code\CBCT_Reconstruction_ICP\data\dataset\count_arch\labels\train.cache



val: Scanning D:\Code\CBCT_Reconstruction_ICP\data\dataset\count_arch\labels\val... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<00:00, 858.46it/s]

val: New cache created: D:\Code\CBCT_Reconstruction_ICP\data\dataset\count_arch\labels\val.cache


...

    100/100      3.16G     0.7937     0.6828      1.288     0.7945     0.9062      1.287         12        640: 100%|██████████| 8/8 [00:00<00:00,  8.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.57it/s]

                   all         31         31      0.958      0.839      0.972      0.796



100 epochs completed in 0.046 hours.
Optimizer stripped from D:\Code\CBCT_Reconstruction_ICP\runs\detect\train14\weights\last.pt, 5.8MB
Optimizer stripped from D:\Code\CBCT_Reconstruction_ICP\runs\detect\train14\weights\best.pt, 5.8MB

Validating D:\Code\CBCT_Reconstruction_ICP\runs\detect\train14\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
YOLOv10n summary (fused): 285 layers, 2694806 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.56it/s]


                   all         31         31      0.958      0.839      0.972      0.796
Speed: 0.3ms preprocess, 2.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to D:\Code\CBCT_Reconstruction_ICP\runs\detect\train14


In [ ]:
# Continue training the model for another 50 epochs

import os
import torch
from ultralytics import YOLOv10

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 创建新的 YOLO 模型实例
model = YOLOv10('yolov10n.yaml')

# 加载之前训练好的模型权重
checkpoint_path = '../runs/detect/train10/weights/best.pt'
model.load(checkpoint_path)

# 确保模型使用 GPU（如果可用）
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("using device: ", device)
model.to(device)

# 使用 'arch_detection.yaml' 数据集继续训练模型，例如再训练 50 个 epochs
results = model.train(data='arch_detection.yaml', epochs=50, device=device)

# 保存继续训练后的模型
model.save('continued_trained_model.pt')

In [14]:
import os
from ultralytics import YOLOv10
import cv2

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Train the model using the 'coco8.yaml' dataset for 3 epochs
model = YOLOv10('../runs/detect/train14/weights/best.pt')

# 定义输入和输出目录
test_images_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/03/adaptive_threshold_jpg'
output_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/03/adaptive_threshold_jpg_predict'

# 创建输出目录（如果不存在）
os.makedirs(output_dir, exist_ok=True)

# 获取测试集中的所有图像文件
test_images = [f for f in os.listdir(test_images_dir) if os.path.isfile(os.path.join(test_images_dir, f))]
positive_indices = []

def plot_predictions(img, preds, output_path):
    for pred in preds:
        x1, y1, x2, y2 = map(int, pred['bbox'])
        label = f"{pred['label']} {pred['confidence']:.2f}"
        print(label)

        # 绘制矩形框
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)

        # 绘制标签
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 保存绘制好预测结果的图像
    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

for index, img_name in enumerate(test_images):
    img_path = os.path.join(test_images_dir, img_name)
    img = cv2.imread(img_path)  # 使用 OpenCV 读取图像
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # 转换为 RGB 格式
    results = model(img_path)  # 对图片进行预测
    preds = []
    has_box = False
    for result in results:
        for box in result.boxes:
            confidence = box.conf[0].item()  # 获取置信度
            if (confidence <= 0.40):
                print('Skipped box with confidence: {} in image {}'.format(confidence, img_name))
                continue
            has_box = True
            bbox = box.xyxy[0].tolist()  # 获取 bbox 的 [x1, y1, x2, y2] 坐标
            label = box.cls[0].item()  # 获取标签
            pred = {
                'bbox': bbox,
                'label': label,
                'confidence': confidence
            }
            preds.append(pred)

    if has_box:
        plot_predictions(img, preds, os.path.join(output_dir, img_name))  # 保存带框的图片
        positive_indices.append(index+1)

print("所有测试图片的预测结果已保存到", output_dir)
print("其中有", len(positive_indices), "张图片中检测到了目标物体")
print("这些图片的索引分别是：", positive_indices)



image 1/1 D:\Code\CBCT_Reconstruction_ICP\data\03\adaptive_threshold_jpg\03_slice_0001.jpg: 640x640 (no detections), 13.2ms
Speed: 3.5ms preprocess, 13.2ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Code\CBCT_Reconstruction_ICP\data\03\adaptive_threshold_jpg\03_slice_0002.jpg: 640x640 (no detections), 12.8ms
Speed: 3.6ms preprocess, 12.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Code\CBCT_Reconstruction_ICP\data\03\adaptive_threshold_jpg\03_slice_0003.jpg: 640x640 (no detections), 13.5ms
Speed: 2.6ms preprocess, 13.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Code\CBCT_Reconstruction_ICP\data\03\adaptive_threshold_jpg\03_slice_0004.jpg: 640x640 (no detections), 13.0ms
Speed: 3.2ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Code\CBCT_Reconstruction_ICP\data\03\adaptive_threshold_jpg\03_slice_0005.jpg: 640x640 (no detecti

In [2]:
# Adaptive threshold filtering

import os
import cv2

def apply_adaptive_threshold(input_dir, output_dir):
    # 创建输出目录（如果不存在）
    os.makedirs(output_dir, exist_ok=True)

    # 获取输入目录中的所有图像文件
    image_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]
    
    for img_name in image_files:
        input_path = os.path.join(input_dir, img_name)
        output_path = os.path.join(output_dir, img_name)
        
        # 读取图像
        image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)  # 读取为灰度图像
        if image is None:
            print(f"无法读取图像: {input_path}")
            continue
        
        # 应用自适应阈值处理
        adaptive_thresh = cv2.adaptiveThreshold(
            image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
            cv2.THRESH_BINARY_INV, 11, 5
        )
        
        # 保存处理后的图像
        cv2.imwrite(output_path, adaptive_thresh)
        print(f"已处理并保存图像: {output_path}")

# 定义输入和输出目录
input_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/01/jpg'
output_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg'

# 应用自适应阈值处理
apply_adaptive_threshold(input_dir, output_dir)

print("所有图像处理完毕。")

已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0001.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0002.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0003.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0004.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0005.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0006.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0007.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0008.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0009.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0010.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/01/adaptive_threshold_jpg\slice_0011.jpg
已处理并保存图像: D:/Code/CBCT_Reconstru

In [6]:
# Adaptive threshold filtering 并去除短线段与点
import os
import cv2
import numpy as np

def apply_adaptive_threshold_and_filter(input_dir, output_dir):
    # 创建输出目录（如果不存在）
    os.makedirs(output_dir, exist_ok=True)

    # 获取输入目录中的所有图像文件
    image_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]
    
    for img_name in image_files:
        input_path = os.path.join(input_dir, img_name)
        output_path = os.path.join(output_dir, img_name)
        
        # 读取图像
        image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)  # 读取为灰度图像
        if image is None:
            print(f"无法读取图像: {input_path}")
            continue
        
        # 应用自适应阈值处理
        adaptive_thresh = cv2.adaptiveThreshold(
            image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
            cv2.THRESH_BINARY_INV, 17, 2
        )
        
        # 定义形态学操作的内核
        kernel = np.ones((3, 3), np.uint8)
        
        # 进行形态学开运算去除噪点
        morph_open = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel)
        
        # 进行形态学闭运算填充线段间的空隙
        morph_close = cv2.morphologyEx(morph_open, cv2.MORPH_CLOSE, kernel)

        # 连通组件分析去除小的联通组件
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(morph_close, connectivity=8)
        min_area = 200  # 设定最小区域面积阈值
        
        filtered_image = np.zeros_like(morph_close)
        
        for i in range(1, num_labels):  # 从1开始，跳过背景
            if stats[i, cv2.CC_STAT_AREA] >= min_area:
                filtered_image[labels == i] = 255
        
        # 保存处理后的图像
        cv2.imwrite(output_path, filtered_image)
        print(f"已处理并保存图像: {output_path}")

# 定义输入和输出目录
base_dir = 'D:/Code/CBCT_Reconstruction_ICP/data/04/'
input_dir = os.path.join(base_dir, 'jpg')
output_dir = os.path.join(base_dir, 'adaptive_threshold_jpg')

# 应用自适应阈值处理和过滤
apply_adaptive_threshold_and_filter(input_dir, output_dir)

print("所有图像处理完毕。")

已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0001.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0002.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0003.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0004.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0005.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0006.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0007.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0008.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0009.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0010.jpg
已处理并保存图像: D:/Code/CBCT_Reconstruction_ICP/data/04/adaptive_threshold_jpg\04_slice_0011.jpg

['AABBTreePolyline2', 'AABBTreePolyline3', 'AffineXf2d', 'AffineXf2f', 'AffineXf3d', 'AffineXf3f', 'BitSet', 'BooleanOperation', 'BooleanResMapObj', 'BooleanResMaps', 'BooleanResult', 'BooleanResultMapper', 'BoostBitSet', 'Box2d', 'Box2f', 'Box3d', 'Box3f', 'Color', 'Contour2d', 'Contour2f', 'Contour3d', 'Contour3f', 'ContourToDistanceMapParams', 'Contours2d', 'Contours2f', 'Contours3d', 'Contours3f', 'DecimateResult', 'DecimateSettings', 'DecimateStrategy', 'DistanceMap', 'DistanceMapToWorld', 'EdgeBitSet', 'EdgeId', 'EdgeMap', 'EdgeMetric', 'EdgePoint', 'ExpectedVoid', 'FaceBitSet', 'FaceFace', 'FaceHashMap', 'FaceId', 'FaceIncidence', 'FaceMap', 'FaceNormals', 'FillHoleMetric', 'FillHoleParams', 'FillHoleParamsMultipleEdgesResolveMode', 'FindOverhangsSettings', 'FixSelfIntersectionMethod', 'FixSelfIntersectionSettings', 'FloatGrid', 'GeneralOffsetParameters', 'GeneralOffsetParametersMode', 'GeodesicPathApprox', 'GridType', 'HolesVertIds', 'ICP', 'ICPMethod', 'ICPMode', 'ICPProperties', 'InflateSettings', 'IntersectionPrecomputesd', 'IntersectionPrecomputesf', 'ItemsView[FaceHashMap]', 'ItemsView[VertHashMap]', 'ItemsView[WholeEdgeHashMap]', 'KeysView[FaceHashMap]', 'KeysView[VertHashMap]', 'KeysView[WholeEdgeHashMap]', 'LaplacianEdgeWeightsParam', 'Line3d', 'Line3f', 'LineSegm2f', 'LineSegm3f', 'LoadDCMResult', 'LoadDCMResults', 'LoadingTiffSettings', 'Matrix2d', 'Matrix2f', 'Matrix3d', 'Matrix3f', 'Mesh', 'MeshApproxRelaxParams', 'MeshBuilderSettings', 'MeshIntersectionResult', 'MeshLoadSettings', 'MeshPart', 'MeshProjectionResult', 'MeshRelaxParams', 'MeshSignedDistanceResult', 'MeshToDistanceMapParams', 'MeshToDistanceVolumeParams', 'MeshToVolumeParams', 'MeshToVolumeParamsType', 'MeshTopology', 'MeshTriPoint', 'MovementBuildBodyParams', 'NestedComponenetsMode', 'OffsetParameters', 'PackMapping', 'PartMapping', 'Path', 'Plane3f', 'PointCloud', 'PointOnFace', 'PointPair', 'PointPairs', 'PointsToMeshParameters', 'Polyline2', 'Polyline3', 'PolylineTopology', 'RelaxApproxType', 'RelaxParams', 'RemeshSettings', 'SceneObject', 'SegmPointf', 'SharpOffsetParameters', 'SignDetectionMode', 'SignedDistanceToMeshResult', 'SimpleVolume', 'SlicePlane', 'SphereParams', 'StitchHolesParams', 'SubdivideSettings', 'SurfacePath', 'SurfacePaths', 'SymbolMeshParams', 'TextAlignParams', 'ThreeVertIds', 'TriPointf', 'Triangulation', 'TriangulationParameters', 'UndirectedEdgeBitSet', 'UndirectedEdgeId', 'UndirectedEdgeMap', 'UndirectedEdgeUndirectedEdge', 'UniteManyMeshesParams', 'ValuesView[FaceHashMap]', 'ValuesView[VertHashMap]', 'ValuesView[WholeEdgeHashMap]', 'VdbVolume', 'Vector2d', 'Vector2f', 'Vector2i', 'Vector3d', 'Vector3f', 'Vector3i', 'VectorFloatByVert', 'VertBitSet', 'VertColorMap', 'VertCoords', 'VertCoords2', 'VertHashMap', 'VertId', 'VertMap', 'ViewportId', 'ViewportMask', 'VoxelsSaveSavingSettings', 'WholeEdgeHashMap', 'WholeEdgeMap', 'WindingMode', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'addBaseToPlanarMesh', 'alignTextToMesh', 'angle', 'boolean', 'buildBottom', 'buildCylinderBetweenTwoHoles', 'buildShortestPath', 'buildShortestPathBiDir', 'buildSmallestMetricPath', 'buildSmallestMetricPathBiDir', 'calcOrientedArea', 'computeDistanceMapD', 'comnces', 'computeThicknessAtVertices', 'copyMesh', 'createSymbolsMesh', 'cross', 'cutAlongEdgeLoop', 'cutMeshWithPlane', 'decimateMesh', 'detectBasisTunnels', 'detectTunnelFaces', 'dilateRegion', 'dilateRegionByMetric', 'distanceMapTo2DIsoPolyline', 'distanceMapToMesh', 'dot', 'doubleOffsetMesh', 'edgeCurvMetric', 'edgeLengthMetric', 'edgeTableSymMetric', 'erodeRegion', 'erodeRegionByMetric', 'expand', 'extractPlaneSections', 'fillContourLeft', 'fillContourLeftByGraphCut', 'fillHole', 'findCollidingTriangles', 'findDegenerateFaces', 'findHoleVertIdsByHoleEdges', 'findLeftBoundary', 'findMaxDistanceSq', 'findMaxDistanceSqOneWay', 'findOverhangs', 'findProjection', 'findRightBoundary', 'findSelfCollidingEdges', 'findSelfCollidingTriangles', 'findSelfCollidingTrianglesBS', 'findSignedDistance', 'findUndercuts', 'fixMultipleEdges', 'fixSelfIntersections', 'fixUndercuts', 'floatGridToVdbVolume', 'generalOffsetMesh', 'getAllComponents', 'getAllComponentsVerts', 'getCircumscribedMetric', 'getComplexFillMetric', 'getComponent', 'getComponentVerts', 'getEdgeLengthFillMetric', 'getEdgeLengthStitchMetric', 'getFacesByMinEdgeLength', 'getIncidentFaces', 'getIncidentVerts', 'getInnerFaces', 'getInnerVerts', 'getLargestComponent', 'getLargestComponentVerts', 'getMinAreaMetric', 'getPlaneFillMetric', 'getUniversalMetric', 'gridToMesh', 'hasMultipleEdges', 'identityMetric', 'inflate', 'intersection', 'loadDCMFolder', 'loadDCMsFolder', 'loadDistanceMapFromImage', 'loadLines', 'loadMesh', 'loadPoints', 'loadSceneObject', 'loadTiffDir', 'loadVoxels', 'localFindSelfIntersections', 'localFixSelfIntersections', 'makeBridge', 'makeBridgeEdge', 'makeConvexHull', 'makeCube', 'makeCylinder', 'makeCylinderAdvanced', 'makeDegenerateBandAroundRegion', 'makeMovementBuildBody', 'makeOuterHalfTorus', 'makeSphere', 'makeTorus', 'makeTorusWithComponents', 'makeTorusWithSelfIntersections', 'makeTorusWithSpikes', 'makeTorusWithUndercut', 'makeUVSphere', 'mergeMeshes', 'meshRegularMap', 'meshToDistanceField', 'meshToDistanceVolume', 'meshToLevelSet', 'meshToPointCloud', 'meshToVolume', 'mixed', 'mult', 'offsetMesh', 'planeSectionsToContours2f', 'pointGridSampling', 'pointIterativeSampling', 'pointUniformSampling', 'pointsToMeshFusion', 'positionVertsSmoothly', 'positionVertsSmoothlySharpBd', 'projectAllMeshVertices', 'rayMeshIntersect', 'relax', 'relaxApprox', 'relaxKeepVolume', 'remesh', 'removeSpikes', 'saveAllSlicesToImage', 'saveDistanceMapToImage', 'saveLines', 'saveMesh', 'savePoints', 'saveSceneObject', 'saveSliceToImage', 'saveVoxels', 'segmentByGraphCut', 'shrink', 'simpleVolumeToDenseGrid', 'simpleVolumeToVdbVolume', 'smoothRegionBoundary', 'stitchContours', 'straightenBoundary', 'subdivideMesh', 'suggestVoxelSize', 'surfacePathLength', 'surroundingContour', 'thickenMesh', 'topologyFromTriangles', 'transformed', 'triangulateContours', 'triangulatePointCloud', 'uniteCloseVertices', 'uniteManyMeshes', 'vdbVolumeToSimpleVolume', 'vectorColor', 'vectorConstMeshPtr', 'vectorEdgePath', 'vectorEdges', 'vectorFaceBitSet', 'vectorFaceFace', 'vectorFaces', 'vectorFloat', 'vectorICPPointPair', 'vectorMesh', 'vectorThreeVertIds', 'vectorUndirectedEdgeUndirectedEdge', 'vectorUndirectedEdges', 'vectorVertBitSet', 'vectorVerts', 'voxelBooleanIntersect', 'voxelBooleanSubtract', 'voxelBooleanUnite']